In [1]:
parameter_file = '../parameters/params.yaml'

import yaml
import sys
import glob
import os
from os.path import join, basename
import pandas as pd
import re

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

print(doc)
gtf = doc["annotation_gtf"]
ref_fa = doc["ref_fa"]

{'data_folder': '/data/isshamie/TSS/NCBI_PICR_103_Processed/', 'results': '/data/isshamie/TSS/Analysis/Analysis_03302020/Results/', 'RNAseq_QC': '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/2017_12_15_hamster_Seq/fastqc/', 'RNAseq_folder': '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/2017_12_15_hamster_Seq/', 'pipeline_path': '/data/isshamie/TSS/Analysis/src/', 'tissues': 'BMDM1hKLA,BMDMwt,Brain,FemaleReproductive,Heart,Kidney,Liver,Lung,MiscOrgans,Muscle,Pancreas,Spleen,CHO', 'ref_fa': '/data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna', 'annotation': '/data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3', 'annotation_gtf': '/data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf', 'mRNA_peak_file': '/data/isshamie/genome/ncbi_anno_103/mRNA_final.peak', 'mRNA_peak_150_file': '/data/isshamie/genome/ncbi_anno_103/mRNA_final_150.peak', 'tss_annotation': '/data/isshamie/genome/ncbi_anno_103/start_site_mRNA_final.tsv', 'cds_anno

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  if sys.path[0] == '':


In [2]:
homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer

/data2/resources/software/homer/v4.11_10242019/bin//homer


In [3]:
file_path = doc["data_folder"]

qc_dir = join(file_path,"QC")
multiqc_dir = join(file_path,"multiqc")

trim_qc_dir = join(file_path,"trim_QC")
trim_multiqc_dir = join(file_path,"trim_multiqc")

bam_dir = join(file_path, "RNAseq","sortBam")
# ht_dir = join(file_path, 'htseq')
tag_dir = join(file_path, "RNAseq",'tags')
# RNA_SeQC = join(file_path,'RNA_SeQC')

results = doc["results"]
csRNA_dir = join(results, 'csRNA_peaks')
if not os.path.exists(csRNA_dir):
    os.mkdir(csRNA_dir)

## Read in metasamples

In [4]:
meta = pd.read_csv('../../params/raw_metaSamples.tsv', sep="\t", index_col=0)
meta.head()


,Tissue,Experiment,Name,Full
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC1,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC2,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,BMDM1hKLA,GRO,BMDM1hKLA_GRO1,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,BMDM1hKLA,GROCap,BMDM1hKLA_GROCap1,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz,BMDMwt,ATAC,BMDMwt_ATAC1,CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_...


## Add in RNA metasamples for Tissues (save in Results?)

In [5]:
rna_files = glob.glob(bam_dir+"/*.bam")
rna_files

['/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL10_Misc_S10_L006_R1_001.fastq.sorted.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL4_Lung_S4_L006_R1_001.fastq.sort.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL7_Brain2_S7_L006_R1_001.fastq.sort.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL6_Heart_S6_L006_R1_001.fastq.sort.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL10_Misc_S10_L006_R1_001.fastq.sort.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL3_Spleen_S3_L006_R1_001.fastq.sort.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL8_Muscle_S8_L006_R1_001.fastq.sort.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL11_Brain7-1_S11_L006_R1_001.fastq.sort.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL1_Pancreas_S1_L006_R1_001.fastq.sort.bam',
 '/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL2_Repr

In [6]:
meta_new = meta.copy()
for curr_f in rna_files:
    if "tmp" in curr_f:
        continue
    sid = "_".join(basename(curr_f).split("_")[1:3])
    tissue = re.sub(r'[^a-zA-Z_]', '', basename(curr_f).split("_")[1])
    name = sid.split("_")[0]
    curr = pd.DataFrame(pd.Series({"Experiment":"RNASeq", "Tissue":tissue, "Full":curr_f, "Name":sid}, name=curr_f)).transpose()
    meta_new = pd.concat((meta_new,curr))

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [7]:
meta_new

,Experiment,Full,Name,Tissue
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...,BMDM1hKLA_ATAC1,BMDM1hKLA
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...,BMDM1hKLA_ATAC2,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,GRO,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...,BMDM1hKLA_GRO1,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,GROCap,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...,BMDM1hKLA_GROCap1,BMDM1hKLA
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz,ATAC,CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_...,BMDMwt_ATAC1,BMDMwt
...,...,...,...,...
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL11_Brain7-1_S11_L006_R1_001.fastq.sort.bam,RNASeq,/data/isshamie/TSS/NCBI_PICR_103_Processed/RNA...,Brain7-1_S11,Brain
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL1_Pancreas_S1_L006_R1_001.fastq.sort.bam,RNASeq,/data/isshamie/TSS/NCBI_PICR_103_Processed/RNA...,Pancreas_S1,Pancreas
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL2_ReproductiveTract_S2_L006_R1_001.fastq.sort.bam,RNASeq,/data/isshamie/TSS/NCBI_PICR_103_Processed/RNA...,ReproductiveTract_S2,ReproductiveTract
/data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/sortBam/NL9_Kidney_S9_L006_R1_001.fastq.sort.bam,RNASeq,/data/isshamie/TSS/NCBI_PICR_103_Processed/RNA...,Kidney_S9,Kidney


## 01
## Need to remake tag directories for csRNA and input with fragment length 150 nt
cmd = "makeTagDirectory {tag_dir} {bam_f} -single -genome {ref_fa} -
checkGC -fragLength 150 

In [8]:
tag_dir_150 = join(results, "tags_150")
if not os.path.exists(tag_dir_150):
    os.mkdir(tag_dir_150)

start_tag = join(tag_dir_150,'START')
if not os.path.exists(start_tag):
    os.mkdir(start_tag)
    

csinput_tag = join(tag_dir_150,'STARTinput')
if not os.path.exists(csinput_tag):
    os.mkdir(csinput_tag)
    

In [9]:
for tis, val in meta_new[meta_new["Experiment"].isin(['START', 'STARTinput', "RNASeq"])].groupby("Tissue"):
    print(tis)
    for start, start_df in val[val["Experiment"].isin(["START"])].iterrows():
        bam_f = join(file_path, tis, "START", "f02_bam", "trim_"+start_df["Name"]+".bam")
        #print(bam_f)
    
        curr_tag = join(start_tag, start_df["Name"])
        #print(start_f)
        cmd = f"makeTagDirectory {curr_tag} {bam_f} -single -genome {ref_fa} -checkGC -fragLength 150 "
        print(cmd)
        os.system(cmd)
    for csinput, csinput_df in val[val["Experiment"].isin(["STARTinput"])].iterrows():
        bam_f = join(file_path, tis, "START", "f02_bam", "trim_"+csinput_df["Name"]+".bam")
        input_tag = join(csinput_tag, csinput_df["Name"]) #join(file_path, tis, "START", "f03_tags", "trim_"+ csinput_df["Name"])
        cmd = f"makeTagDirectory {input_tag} {bam_f} -single -genome {ref_fa} -checkGC -fragLength 150"
        print(cmd)
        os.system(cmd)
    

Brain
makeTagDirectory /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Brain_START1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f02_bam/trim_Brain_START1.bam -single -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC -fragLength 150 
makeTagDirectory /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Brain_START2 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f02_bam/trim_Brain_START2.bam -single -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC -fragLength 150 
makeTagDirectory /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Brain_START3 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f02_bam/trim_Brain_START3.bam -single -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC -fragLength 150 
makeTagDirectory /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/STARTinp

Muscle
makeTagDirectory /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Muscle_START1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Muscle/START/f02_bam/trim_Muscle_START1.bam -single -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC -fragLength 150 
makeTagDirectory /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Muscle_START2 /data/isshamie/TSS/NCBI_PICR_103_Processed/Muscle/START/f02_bam/trim_Muscle_START2.bam -single -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC -fragLength 150 
makeTagDirectory /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/STARTinput/Muscle_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Muscle/START/f02_bam/trim_Muscle_STARTinput1.bam -single -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -checkGC -fragLength 150
Pancreas
makeTagDirectory /data/isshamie/TSS/Analysis/Analysis_0

## 02
## a. findcsRNATSSS

In [10]:
for tis, val in meta_new[meta_new["Experiment"].isin(['START', 'STARTinput', "RNASeq"])].groupby("Tissue"):
    print(tis)
    for start, start_df in val[val["Experiment"].isin(["START"])].iterrows():    
        start_f = join(start_tag, start_df["Name"])
        for csinput, csinput_df in val[val["Experiment"].isin(["STARTinput"])].iterrows():
            inp_f = join(csinput_tag, csinput_df["Name"])
            
            for rna, rna_df in val[val["Experiment"].isin(["RNASeq"])].iterrows():
                rna_f = glob.glob(join(tag_dir, "*" + rna_df["Name"].split("-")[0]))[0]

                out_name = '%s_%s_%s_%s' % (tis,start_df["Name"], csinput_df["Name"], rna_df["Name"])
                out_f = os.path.join(csRNA_dir,out_name) #'Results/csRNATSS/%s_%s_%s' % (t,os.path.basename(i),os.path.basename(j))
                cmd = f'findcsRNATSS.pl {start_f} -i {inp_f} -rna {rna_f} -gtf {gtf} -genome {ref_fa} -cpu 30 -o {out_f}'
                print(cmd)
                os.system(cmd)

Brain
findcsRNATSS.pl /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Brain_START1 -i /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/STARTinput/Brain_STARTinput1 -rna /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Brain2_S7 -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome ch -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Brain_Brain_START1_Brain_STARTinput1_Brain2_S7
findcsRNATSS.pl /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Brain_START1 -i /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/STARTinput/Brain_STARTinput1 -rna /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Brain7 -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome ch -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Brain_Brain_START1_Brain_STARTinput1_Brain7-1_S11
findcsRNATSS.pl /data/isshamie/TSS/Analysis/Analysis_033

Lung
findcsRNATSS.pl /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Lung_START1 -i /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/STARTinput/Lung_STARTinput1 -rna /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/test_Lung_S4 -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome ch -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Lung_Lung_START1_Lung_STARTinput1_Lung_S4
findcsRNATSS.pl /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Lung_START2 -i /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/STARTinput/Lung_STARTinput1 -rna /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/test_Lung_S4 -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome ch -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Lung_Lung_START2_Lung_STARTinput1_Lung_S4
Misc
MiscOrgans
Muscle
findcsRNATSS.pl /data/isshamie/TSS/Analysi

## b. Specifying RNASeq 150 bp tag directory

In [14]:
join(rna_tag_dir+"_frag150", "*" + rna_df["Name"].split("-")[0])

'/data/isshamie/TSS/NCBI_PICR_103_Processed/tags_frag150/*Brain2_S7'

In [16]:
rna_tag_dir = join(file_path, 'tags')

for tis, val in meta_new[meta_new["Experiment"].isin(['START', 'STARTinput', "RNASeq"])].groupby("Tissue"):
    print(tis)
    for start, start_df in val[val["Experiment"].isin(["START"])].iterrows():    
        start_f = join(start_tag, start_df["Name"])
        for csinput, csinput_df in val[val["Experiment"].isin(["STARTinput"])].iterrows():
            inp_f = join(csinput_tag, csinput_df["Name"])
            
            for rna, rna_df in val[val["Experiment"].isin(["RNASeq"])].iterrows():
                rna_f = join(rna_tag_dir, rna_df["Name"].split("-")[0] + "_frag150")
                rna_f = glob.glob(rna_f)[0]
            
                out_name = '%s_%s_%s_%s' % (tis,start_df["Name"], csinput_df["Name"], rna_df["Name"])
                out_f = os.path.join(csRNA_dir,out_name) #'Results/csRNATSS/%s_%s_%s' % (t,os.path.basename(i),os.path.basename(j))
                cmd = f'findcsRNATSS.pl {start_f} -i {inp_f} -rna {rna_f} -gtf {gtf} -genome {ref_fa} -cpu 30 -o {out_f}'
                print(cmd)
                os.system(cmd)
                join(tag_dir,sample_id+"_frag150")

Brain
findcsRNATSS.pl /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/START/Brain_START1 -i /data/isshamie/TSS/Analysis/Analysis_03302020/Results/tags_150/STARTinput/Brain_STARTinput1 -rna [] -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Brain_Brain_START1_Brain_STARTinput1_Brain2_S7


NameError: name 'sample_id' is not defined

In [9]:
# for tis, val in meta_new[meta_new["Experiment"].isin(['START', 'STARTinput', "RNASeq"])].groupby("Tissue"):
#     print(tis)
#     for start, start_df in val[val["Experiment"].isin(["START"])].iterrows():
#         start_f = join(file_path, tis, "START", "f03_tags", "trim_"+start_df["Name"])
#         #print(start_f)
#         for csinput, csinput_df in val[val["Experiment"].isin(["STARTinput"])].iterrows():
#             inp_f = join(file_path, tis, "START", "f03_tags", "trim_"+ csinput_df["Name"])
            
#             for rna, rna_df in val[val["Experiment"].isin(["RNASeq"])].iterrows():
#                 rna_f = glob.glob(join(tag_dir, "*" + rna_df["Name"].split("-")[0]))[0]
                
#                 out_name = '%s_%s_%s_%s' % (tis,start_df["Name"], csinput_df["Name"], rna_df["Name"])
#                 out_f = os.path.join(csRNA_dir,out_name) #'Results/csRNATSS/%s_%s_%s' % (t,os.path.basename(i),os.path.basename(j))

#                 cmd = f'findcsRNATSS.pl {start_f} -i {inp_f} -rna {rna_f} -gtf {gtf} -genome ch -cpu 30 -o {out_f}'
#                 print(cmd)
#                 os.system(cmd)
#                 break

Brain
findcsRNATSS.pl /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START1 -i /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_STARTinput1 -rna /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Brain2_S7 -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome ch -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Brain_Brain_START1_Brain_STARTinput1_Brain2_S7
findcsRNATSS.pl /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START1 -i /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_STARTinput2 -rna /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Brain2_S7 -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome ch -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Brain_Brain_START1_Brain_STARTinput2_Brain2_S7
findcsRNATSS.pl /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/S

ReproductiveTract
Skin
Spleen
findcsRNATSS.pl /data/isshamie/TSS/NCBI_PICR_103_Processed/Spleen/START/f03_tags/trim_Spleen_START1 -i /data/isshamie/TSS/NCBI_PICR_103_Processed/Spleen/START/f03_tags/trim_Spleen_STARTinput1 -rna /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Spleen_S3 -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome ch -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Spleen_Spleen_START1_Spleen_STARTinput1_Spleen_S3
findcsRNATSS.pl /data/isshamie/TSS/NCBI_PICR_103_Processed/Spleen/START/f03_tags/trim_Spleen_START2 -i /data/isshamie/TSS/NCBI_PICR_103_Processed/Spleen/START/f03_tags/trim_Spleen_STARTinput1 -rna /data/isshamie/TSS/NCBI_PICR_103_Processed/RNAseq/tags/Spleen_S3 -gtf /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gtf -genome ch -cpu 30 -o /data/isshamie/TSS/Analysis/Analysis_03302020/Results/csRNA_peaks/Spleen_Spleen_START2_Spleen_STARTinput1_Spleen_S3
Total


In [ ]:
# #cmd = 'findcsRNATSS.pl /data/isshamie/TSS/Processed/CHO/mSTART/f03_tags/trim_CHO--mSTART-JHS823_S21_R1_001.fastq -i /data/isshamie/TSS/Processed/CHO/mSTART/f03_tags/trim_CHO--mSTART_input-JHS802_S11_R1_001.fastq -gtf /data/isshamie/genome/updated_final_sort.gtf -genome ch -cpu 30 -o Results/csRNATSS/Example/CHO_trim_CHO--mSTART-JHS823_S21_R1_001.fastq_trim_CHO--mSTART_input-JHS802_S11_R1_001.fastq
# i = tss_tag[0]
# j = inp_tag[0]
# k = rna_tag[0]


# out_name = '%s_%s_%s' % (t,os.path.basename(i),os.path.basename(j))
# out_f = os.path.join(csRNA_dir,out_name) #'Results/csRNATSS/%s_%s_%s' % (t,os.path.basename(i),os.path.basename(j))

# cs_cmd = 'findcsRNATSS.pl {start} -i {control} -rna {rna} -gtf {annotation_gtf} -genome ch -cpu 30 > {out_f}'.format(
#     start=i,
#     control=j,
#     rna=rna_tag,
#     annotation_gtf=annotation_gtf,
#     out_f=out_f)